# CMMN (Convolutional Monge Mapping Normalization) Visualization

This notebook demonstrates the CMMN domain adaptation method for EEG signals.

Dr. B has recommended to do the notebook here in colab so that we all have the same data and can easily run it. I think it's a good idea so I'm porting it over now. Still saving to the git repo though.

Later, once this is complete, will refresh on the newly refactored codebase to do BoW on this and then Carlos can test it with his shiny new clfs.

Right now this is finished, currently setting up a Caviness job to run through everything.

Post NER note - moved to notebooks, make sure that the filepaths work correctly when re-visualized.

In [1]:
# Set OMP constants to use only 8 CPUs
import os
os.environ["OMP_NUM_THREADS"] = "8"

In [2]:
# Imports
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import loadmat

# Add parent directory to path for icwaves imports
sys.path.append('..')

# Import CMMN functions from the new module
from icwaves.cmmn import (
    psd,
    compute_normed_barycenter,
    compute_filter_original,
    compute_filter_subj_subj,
    subj_subj_matching,
    transform_original,
    transform_data_subj_subj,
    transform_data_subj_subj_single,
    transform_original_single,
    plot_psd,
    plot_raw_signals,
    plot_polysomnograph,
    plot_barycenter,
    plot_freq_filter,
    plot_time_filter,
)

## Setup Paths and Parameters

In [3]:
# Full subject lists for HPC processing
emotion_subj_list = [f'{i:02d}' for i in range(1, 36) if i != 22]  # 01-35 minus 22
frolich_subj_list = [f'{i:02d}' for i in range(1, 13)]  # 01-12

# Output directory for PDF figures
output_dir = Path('./ner_figures')
output_dir.mkdir(parents=True, exist_ok=True)

# Flag for PSD computation
make_psds = False  # change to True if running the notebook for the first time.

In [4]:
# HPC file paths
data_dir = Path('../data')
emotion_filepath = data_dir / 'emotion_256' / 'raw_data_and_IC_labels'
frolich_filepath = data_dir / 'frolich_256' / 'frolich_extract_256_hz'

emotion_transformed_filepath = data_dir / 'emotion_256' / 'emotion_data_cmmn_ch_avg_barycenter'
frolich_transformed_filepath = data_dir / 'frolich_256' / 'frolich_extract_256_hz_cmmn_ch_avg_barycenter'
emotion_transformed_psd_filepath = data_dir / 'emotion_256' / 'emotion_data_cmmn_ch_avg_barycenter' / 'psds'
frolich_transformed_subj_subj_psd_filepath = data_dir / 'frolich_256' / 'frolich_extract_256_hz_cmmn_ch_avg_barycenter' / 'subj_subj_psds'
frolich_transformed_original_psd_filepath = data_dir / 'frolich_256' / 'frolich_extract_256_hz_cmmn_ch_avg_barycenter' / 'original_psds'

# PSD file paths  
emotion_normed_psd_filepath = emotion_filepath / 'psds_normed'
frolich_normed_psd_filepath = frolich_filepath / 'psds_normed'

emotion_raw_psd_filepath = emotion_filepath / 'psds'
frolich_raw_psd_filepath = frolich_filepath / 'psds'

# Create all necessary directories
emotion_transformed_filepath.mkdir(parents=True, exist_ok=True)
frolich_transformed_filepath.mkdir(parents=True, exist_ok=True)
emotion_transformed_psd_filepath.mkdir(parents=True, exist_ok=True)
frolich_transformed_subj_subj_psd_filepath.mkdir(parents=True, exist_ok=True)
frolich_transformed_original_psd_filepath.mkdir(parents=True, exist_ok=True)

## Load Raw Data

In [5]:
# Load raw data
emotion_data = []
for subj in emotion_subj_list:
    emotion_data.append(loadmat(emotion_filepath / f'subj-{subj}.mat')['data'])

frolich_data = []
for subj in frolich_subj_list:
    frolich_data.append(loadmat(frolich_filepath / f'frolich_extract_{subj}_256_hz.mat')['X'])

print(f"Loaded {len(emotion_data)} emotion subjects and {len(frolich_data)} Fröhlich subjects")

KeyboardInterrupt: 

## Compute or Load PSDs

In [ ]:
# Compute PSDs if needed
if make_psds:
    emotion_raw_psd_filepath.mkdir(parents=True, exist_ok=True)
    frolich_raw_psd_filepath.mkdir(parents=True, exist_ok=True)

    for i, subj in enumerate(emotion_data):
        f, Pxx = psd(subj)
        np.savez(emotion_raw_psd_filepath / f'subj-{emotion_subj_list[i]}_psds', Pxx)

    for i, subj in enumerate(frolich_data):
        f, Pxx = psd(subj)
        np.savez(frolich_raw_psd_filepath / f'frolich_extract_{frolich_subj_list[i]}_256_hz_psds', Pxx)
    
    print("PSDs computed and saved")

In [ ]:
# Load PSDs
emotion_data_psds_raw = []
frolich_data_psds_raw = []

emotion_data_psds_normed = []
frolich_data_psds_normed = []

# Load raw PSDs
if (emotion_raw_psd_filepath).exists():
    for subj in emotion_subj_list:
        emotion_data_psds_raw.append(np.load(emotion_raw_psd_filepath / f'subj-{subj}_psds.npz')['arr_0'])

if (frolich_raw_psd_filepath).exists():
    for subj in frolich_subj_list:
        frolich_data_psds_raw.append(np.load(frolich_raw_psd_filepath / f'frolich_extract_{subj}_256_hz_psds.npz')['arr_0'])

# Load normed PSDs
if emotion_normed_psd_filepath.exists():
    for subj in emotion_subj_list:
        emotion_data_psds_normed.append(np.load(emotion_normed_psd_filepath / f'subj-{subj}_psds_normed.npz')['arr_0'])

if frolich_normed_psd_filepath.exists():
    for subj in frolich_subj_list:
        frolich_data_psds_normed.append(np.load(frolich_normed_psd_filepath / f'frolich_extract_{subj}_256_hz_psds_normed.npz')['arr_0'])

print(f"Loaded {len(emotion_data_psds_raw)} emotion PSDs and {len(frolich_data_psds_raw)} Fröhlich PSDs")

## Load Filters

In [ ]:
# HPC filter paths
filters_filepath = data_dir / 'frolich_filters'

emotion_original_time_filters = []
emotion_original_freq_filters = []

for subj in emotion_subj_list:  # Load filters for all subjects
    time_filter_path = filters_filepath / f'emotion_normed_psds_normed_barycenter_ch_avg_barycenter_time_filter_{subj}.npz'
    freq_filter_path = filters_filepath / f'emotion_normed_psds_normed_barycenter_ch_avg_barycenter_freq_filter_{subj}.npz'
    
    if time_filter_path.exists():
        emotion_original_time_filters.append(np.load(time_filter_path)['arr_0'])
    if freq_filter_path.exists():
        emotion_original_freq_filters.append(np.load(freq_filter_path)['arr_0'])

frolich_original_time_filters = []
frolich_original_freq_filters = []
frolich_subj_subj_freq_filters = []
frolich_subj_subj_time_filters = []

for subj in frolich_subj_list:
    frolich_original_time_filters.append(np.load(filters_filepath / f'frolich_original_ch_avg_barycenter_time_filter_{subj}.npz')['arr_0'])
    frolich_original_freq_filters.append(np.load(filters_filepath / f'frolich_original_ch_avg_barycenter_freq_filter_{subj}.npz')['arr_0'])
    frolich_subj_subj_freq_filters.append(np.load(filters_filepath / f'frolich_subj_subj_ch_avg_barycenter_freq_filter_{subj}.npz')['arr_0'])
    frolich_subj_subj_time_filters.append(np.load(filters_filepath / f'frolich_subj_subj_ch_avg_barycenter_time_filter_{subj}.npz')['arr_0'])

print(f"Loaded filters for {len(emotion_original_time_filters)} emotion subjects and {len(frolich_original_time_filters)} Fröhlich subjects")

## Visualize Filters

In [ ]:
# Visualize emotion filters
if emotion_original_time_filters:
    plot_time_filter(emotion_original_time_filters, 
                    title='Emotion Barycenter Mapping Time Filters', 
                    save_path=output_dir / 'emotion_original_time_filters.pdf')

if emotion_original_freq_filters:
    plot_freq_filter(emotion_original_freq_filters, 
                    title='Emotion Barycenter Mapping Frequency Filters', 
                    save_path=output_dir / 'emotion_original_freq_filters.pdf')

In [ ]:
# Visualize Fröhlich filters
if frolich_original_time_filters:
    plot_time_filter(frolich_original_time_filters, 
                    title='Cue Barycenter Mapping Time Filters', 
                    save_path=output_dir / 'frolich_original_time_filters.pdf')

if frolich_original_freq_filters:
    plot_freq_filter(frolich_original_freq_filters, 
                    title='Cue Barycenter Mapping Frequency Filters', 
                    save_path=output_dir / 'frolich_original_freq_filters.pdf')

# Also plot Fröhlich subject-to-subject filters
if frolich_subj_subj_time_filters:
    plot_time_filter(frolich_subj_subj_time_filters, 
                    title='Cue Subj-to-subj Mapping Time Filters', 
                    save_path=output_dir / 'frolich_subj_subj_time_filters.pdf')

if frolich_subj_subj_freq_filters:
    plot_freq_filter(frolich_subj_subj_freq_filters, 
                    title='Cue Subj-to-subj Mapping Frequency Filters', 
                    save_path=output_dir / 'frolich_subj_subj_freq_filters.pdf')

## Visualize Raw PSDs and Barycenter

In [ ]:
# Raw PSDs for emotion and Fröhlich
plot_psd(emotion_data, psds=emotion_data_psds_raw, 
         title='Emotion Data, Unnormalized PSDs', 
         save_path=output_dir / 'emotion_psd_raw.pdf')

plot_psd(frolich_data, psds=frolich_data_psds_raw, 
         title='Cue Data, Unnormalized PSDs', 
         save_path=output_dir / 'frolich_psd_raw.pdf')

In [ ]:
# Normed barycenter for emotion
normed_emotion_barycenter = np.load(data_dir / 'frolich_filters' / 'emotion_normed_ch_avg_barycenter.npz')['arr_0']
plot_barycenter(normed_emotion_barycenter, 
                title='Normalized Emotion Barycenter', 
                save_path=output_dir / 'emotion_normed_barycenter.pdf')

## Transform Data Using Filters

In [ ]:
# Transform the Fröhlich data using the provided filters
for i, subj in enumerate(frolich_subj_list):
    # Check if transformed data already exists
    subj_subj_path = frolich_transformed_filepath / f'frolich_extract_{subj}_256_hz_subj_subj_ch_avg_barycenter_cmmn.npz'
    subj_subj_psd_path = frolich_transformed_subj_subj_psd_filepath / f'frolich_extract_{subj}_256_hz_subj_subj_ch_avg_barycenter_cmmn_psds.npz'
    original_path = frolich_transformed_filepath / f'frolich_extract_{subj}_256_hz_original_ch_avg_barycenter_cmmn.npz'
    original_psd_path = frolich_transformed_original_psd_filepath / f'frolich_extract_{subj}_256_hz_original_ch_avg_barycenter_cmmn_psds.npz'
    
    if subj_subj_path.exists() and subj_subj_psd_path.exists() and original_path.exists() and original_psd_path.exists():
        continue

    # Print progress
    print(f'Transforming Fröhlich data for subject {subj} ({i+1}/{len(frolich_subj_list)})')

    # Subject-to-subject transformation
    time_filter = frolich_subj_subj_time_filters[i]
    transformed_data = transform_data_subj_subj_single(frolich_data[i], time_filter)
    np.savez(subj_subj_path, transformed_data)

    # Save PSDs
    f, Pxx = psd(transformed_data)
    np.savez(subj_subj_psd_path, Pxx)

    # Original filter transformation
    time_filter = frolich_original_time_filters[i]
    transformed_data = transform_original_single(frolich_data[i], time_filter)
    np.savez(original_path, transformed_data)

    # Save PSDs
    f, Pxx = psd(transformed_data)
    np.savez(original_psd_path, Pxx)

    print(f'Transformed Fröhlich data for subject {subj} and calculated and saved PSDs')

In [ ]:
# Transform the emotion data using original filters
for i, subj in enumerate(emotion_subj_list):
    # Skip if already done
    emotion_path = emotion_transformed_filepath / f'emotion_data_{subj}_256_hz_original_ch_avg_barycenter_cmmn.npz'
    emotion_psd_path = emotion_transformed_psd_filepath / f'emotion_data_{subj}_256_hz_original_ch_avg_barycenter_cmmn_psds.npz'
    
    if emotion_path.exists() and emotion_psd_path.exists():
        continue

    # Print progress
    print(f'Transforming emotion data for subject {subj} ({i+1}/{len(emotion_subj_list)})')

    time_filter = emotion_original_time_filters[i]
    transformed_data = transform_original_single(emotion_data[i], time_filter)
    np.savez(emotion_path, transformed_data)

    # Save PSDs
    f, Pxx = psd(transformed_data)
    np.savez(emotion_psd_path, Pxx)

    print(f'Transformed emotion data for subject {subj} and calculated and saved PSDs')

## Visualize Transformed Data

In [ ]:
# Load transformed PSDs
frolich_transformed_subj_subj_psd_filepath.mkdir(parents=True, exist_ok=True)
frolich_transformed_original_psd_filepath.mkdir(parents=True, exist_ok=True)
emotion_transformed_psd_filepath.mkdir(parents=True, exist_ok=True)

frolich_transformed_subj_subj_psds = []
frolich_transformed_original_psds = []
emotion_transformed_psds = []

for subj in frolich_subj_list:
    subj_subj_psd_path = frolich_transformed_subj_subj_psd_filepath / f'frolich_extract_{subj}_256_hz_subj_subj_ch_avg_barycenter_cmmn_psds.npz'
    original_psd_path = frolich_transformed_original_psd_filepath / f'frolich_extract_{subj}_256_hz_original_ch_avg_barycenter_cmmn_psds.npz'
    
    if subj_subj_psd_path.exists():
        frolich_transformed_subj_subj_psds.append(np.load(subj_subj_psd_path)['arr_0'])
    if original_psd_path.exists():
        frolich_transformed_original_psds.append(np.load(original_psd_path)['arr_0'])

for subj in emotion_subj_list:
    emotion_psd_path = emotion_transformed_psd_filepath / f'emotion_data_{subj}_256_hz_original_ch_avg_barycenter_cmmn_psds.npz'
    if emotion_psd_path.exists():
        emotion_transformed_psds.append(np.load(emotion_psd_path)['arr_0'])

print(f"Loaded {len(frolich_transformed_subj_subj_psds)} transformed Fröhlich PSDs and {len(emotion_transformed_psds)} transformed emotion PSDs")

In [ ]:
# Plot the transformed PSDs
if frolich_transformed_subj_subj_psds:
    plot_psd(frolich_transformed_subj_subj_psds, 
             title='Cue Transformed - Subj-to-subj Mapping', 
             save_path=output_dir / 'frolich_transformed_subj_subj_psds.pdf')

if frolich_transformed_original_psds:
    plot_psd(frolich_transformed_original_psds, 
             title='Cue Transformed - Barycenter Mapping', 
             save_path=output_dir / 'frolich_transformed_original_psds.pdf')

if emotion_transformed_psds:
    plot_psd(emotion_transformed_psds, 
             title='Emotion Transformed - Barycenter Mapping', 
             save_path=output_dir / 'emotion_transformed_psds.pdf')

## Summary

In [ ]:
print("\n" + "="*60)
print("VISUALIZATION COMPLETE!")
print("="*60)
print(f"All figures saved to: {output_dir}")
print(f"Processed {len(emotion_subj_list)} emotion subjects and {len(frolich_subj_list)} Fröhlich subjects")

# Print which PSDs were used
print(f"\nPSD Usage:")
if emotion_data_psds_normed:
    print(f"  - Emotion: Using NORMED PSDs for processing")
else:
    print(f"  - Emotion: Using RAW PSDs (normed not found)")

if frolich_data_psds_normed:
    print(f"  - Fröhlich: Using NORMED PSDs for processing")
else:
    print(f"  - Fröhlich: Using RAW PSDs (normed not found)")

print("\nFigures generated:")

for fig_file in sorted(output_dir.glob("*.pdf")):
    print(f"  - {fig_file.name}")

print(f"\nTotal PDF files: {len(list(output_dir.glob('*.pdf')))}")
print("Ready for NER paper submission!")